# An Integrated Macro-Financial Model for Fair Value Estimation

**Abstract:**

This notebook presents a comprehensive, integrated model for equity market valuation that bridges the gap between macroeconomic fundamentals and financial asset pricing. We employ the neoclassical Solow-Swan growth model as the foundational engine for determining long-run real economic growth. The steady-state output from this model is then channeled into a financial valuation framework, which utilizes the Capital Asset Pricing Model (CAPM) to derive the required rate of return on equity and the Gordon Growth Model to calculate a "justified" forward Price-to-Earnings (P/E) ratio.

The analysis extends beyond deterministic point estimates by incorporating interactive sensitivity analyses and a robust Monte Carlo simulation. These tools allow for a nuanced exploration of how key macroeconomic and financial parameters—such as the savings rate, technological progress, and risk premia—influence economic growth and equity valuations. The final section introduces a stochastic framework to model uncertainty, providing a probabilistic distribution of potential valuation outcomes. This work serves as both an educational tool for understanding macro-financial linkages and a practical framework for fundamental market analysis.

**Keywords:** *Solow-Swan Growth Model, Capital Asset Pricing Model (CAPM), Gordon Growth Model, Justified P/E Ratio, Monte Carlo Simulation, Macro-Financial Linkages.*


## 1. Foundational Setup: Libraries and Model Specification

This section imports the necessary Python libraries for numerical computation (`numpy`), data manipulation (`pandas`), visualization (`matplotlib`), and interactive controls (`ipywidgets`). These tools form the foundation upon which our economic and financial models are built.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import interact, FloatSlider
from ipywidgets import Layout


## 2. The Solow-Swan Growth Model: Engine of Economic Growth

This cell defines the core functions of the neoclassical Solow-Swan growth model, which serves as the macroeconomic foundation for our analysis.

### Economic Rationale
The Solow-Swan model (Solow, 1956; Swan, 1956) is a cornerstone of modern macroeconomics that describes long-run economic growth. It posits that an economy's output is determined by its stocks of capital and labor, augmented by technological progress. The model's central equation governs the evolution of the capital stock, which increases with investment (a fraction `s` of output) and decreases with depreciation (`δ`).

The key insight is that, in the long run, the economy converges to a "steady state" where the growth rate of output per effective worker is zero. Consequently, the aggregate economy grows at a rate determined by the sum of population growth (`n`) and the rate of exogenous technological progress (`g`). This steady-state growth rate is the crucial output from our model, as it represents the sustainable, long-term real growth of the economy, which is a fundamental determinant of asset values.

### Functional Specification
- **`next_period_k(...)`**: Implements the capital accumulation equation: \( K_{t+1} = (1-δ)K_t + sY_t \). This function calculates the capital stock in the next period based on current capital, output, the savings rate, and the depreciation rate.
- **`solow_model_final_growth_rate(...)`**: Simulates the economy for a specified number of periods (`T`) to find the terminal growth rate. This function effectively solves for the model's steady-state growth rate, which theoretically converges to `n + g`.
- **`solow_model_simulation_with_growth_rate(...)`**: Runs a full simulation and plots the year-over-year growth rate of output. This visualization allows us to observe the economy's convergence towards its long-run steady-state path.

### References
- Solow, R. M. (1956). A Contribution to the Theory of Economic Growth. *The Quarterly Journal of Economics, 70*(1), 65–94.
- Swan, T. W. (1956). Economic Growth and Capital Accumulation. *Economic Record, 32*(2), 334–361.


In [16]:
# First, make sure you have the required functions
def next_period_k(k, s, y, delta, n, g):
    investment = s * y
    depreciation = delta * k
    next_k = (1 + n + g) * (k + investment - depreciation)
    return next_k

def solow_model_simulation_with_growth_rate(s=0.20, n=0.005, g=0.02, delta=0.05, alpha=0.35, A=1, L=100, K=10000, T=1000):
    # Arrays to store simulation results and growth rates
    K_t = np.zeros(T)
    Y_t = np.zeros(T)
    A_t = np.zeros(T)
    L_t = np.zeros(T)
    growth_rates = np.zeros(T-1)

    # Initial values
    K_t[0] = K
    A_t[0] = A
    L_t[0] = L
    Y_t[0] = (K_t[0]**alpha) * (A_t[0]*L_t[0])**(1-alpha)

    # Run the simulation and calculate growth rates
    for t in range(1, T):
        K_t[t] = next_period_k(K_t[t-1], s, Y_t[t-1], delta, n, g)
        L_t[t] = L_t[t-1] * (1 + n)
        A_t[t] = A_t[t-1] * (1 + g)
        Y_t[t] = (K_t[t]**alpha) * (A_t[t]*L_t[t])**(1-alpha)
        growth_rates[t-1] = (Y_t[t] - Y_t[t-1]) / Y_t[t-1] * 100

    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(np.arange(1, T), growth_rates, label='Output Growth Rate', color='green', linewidth=2)
    plt.title('Output Growth Rate over Time')
    plt.xlabel('Time')
    plt.ylabel('Growth Rate (%)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# CORRECTED WIDGET CODE
# Create widgets with proper styling
s_widget = widgets.FloatSlider(
    value=0.05, min=0.0, max=1.0, step=0.01,
    description='Savings Rate:',
    style={'description_width': 'initial'},
    layout=Layout(width='50%')
)

n_widget = widgets.FloatSlider(
    value=0.01, min=0.0, max=0.1, step=0.001,
    description='Population Growth:',
    readout_format='.3f',
    style={'description_width': 'initial'},
    layout=Layout(width='50%')
)

g_widget = widgets.FloatSlider(
    value=0.02, min=0.0, max=0.1, step=0.001,
    description='Tech Progress:',
    readout_format='.3f',
    style={'description_width': 'initial'},
    layout=Layout(width='50%')
)

delta_widget = widgets.FloatSlider(
    value=0.05, min=0.0, max=0.2, step=0.01,
    description='Depreciation Rate:',
    style={'description_width': 'initial'},
    layout=Layout(width='50%')
)

alpha_widget = widgets.FloatSlider(
    value=0.35, min=0.0, max=1.0, step=0.01,
    description='Capital Elasticity:',
    style={'description_width': 'initial'},
    layout=Layout(width='50%')
)

# Create output widget to capture the plots
output_widget = widgets.Output()

# CORRECTED Update function
def update_model(change):
    """Update function that properly captures output"""
    with output_widget:
        clear_output(wait=True)
        # Call the simulation with current widget values
        solow_model_simulation_with_growth_rate(
            s=s_widget.value,
            n=n_widget.value,
            g=g_widget.value,
            delta=delta_widget.value,
            alpha=alpha_widget.value
        )

# Attach observers to all widgets
s_widget.observe(update_model, names='value')
n_widget.observe(update_model, names='value')
g_widget.observe(update_model, names='value')
delta_widget.observe(update_model, names='value')
alpha_widget.observe(update_model, names='value')

# Create container with all widgets
container = widgets.VBox([
    s_widget,
    n_widget,
    g_widget,
    delta_widget,
    alpha_widget
])

# Display the widgets and output
display(container)
display(output_widget)

# Initial plot with default values
update_model(None)

print("✅ Interactive widgets are now properly configured!")
print("Move the sliders above to see the plot update in real-time.")

Output()

✅ Interactive widgets are now properly configured!
Move the sliders above to see the plot update in real-time.


## 3. Interactive Dashboard for Solow Model Exploration

This cell develops an interactive dashboard using `ipywidgets` to facilitate a real-time, visual exploration of the Solow model's dynamics.

### Pedagogical and Analytical Purpose
The primary purpose of this interactive tool is to demonstrate the comparative statics of the Solow model. By manipulating the sliders, one can instantly observe how changes in the core parameters—the savings rate (`s`), population growth (`n`), technological progress (`g`), depreciation rate (`δ`), and capital elasticity (`α`)—affect the long-run economic growth trajectory.

For instance, a user can test established theoretical propositions:
- **Increasing the savings rate (`s`)**: This raises the level of output per capita but does not affect the long-run growth rate. The plot will show a temporary spike in growth as the economy transitions to a higher steady-state capital stock.
- **Increasing the technological progress rate (`g`)**: This permanently increases the long-run growth rate of the economy. The plot will converge to a new, higher steady-state growth rate.

This hands-on approach provides a more intuitive understanding of the model's mechanics than static analysis alone, making it a valuable tool for both research and educational purposes. The `update_model` function serves as a callback that is triggered upon any change in a widget's value, clearing the previous output and regenerating the plot with the new parameter set.


In [4]:
def plot_potential_gdp(s=0.20, n=0.005, delta=0.05, alpha=0.35, A=1, L=100, K=10000):
    g_range = np.arange(0.01, 0.03, 0.001)
    potential_gdp = [solow_model_final_growth_rate(s=s, n=n, g=g, delta=delta, alpha=alpha, A=A, L=L, K=K,T=100) for g in g_range]

    plt.figure(figsize=(10, 6))
    plt.plot((g_range*100), potential_gdp, label='Potential GDP Growth Rate')
    plt.xlabel('Technological Progress Rate % (g)')
    plt.ylabel('Potential GDP Growth Rate (%)')
    plt.title('Potential GDP Growth Rate vs. Technological Progress Rate')
    plt.legend()
    plt.grid(True)
    plt.show()

# Helper function to create a slider with adjusted description width
def create_slider(min, max, step, value, description, format):
    return widgets.FloatSlider(
        min=min, max=max, step=step, value=value, description=description,
        layout=Layout(width='50%'),
        style={'description_width': 'initial'},  # This is the key change
        readout_format=format,
    )

# Create sliders using the helper function
s_slider = create_slider(0, 0.5, 0.01, 0.20, 'Savings Rate (s):', '.2f')
n_slider = create_slider(0, 0.1, 0.001, 0.005, 'Population Growth Rate (n):', '.3f')
delta_slider = create_slider(0, 0.1, 0.01, 0.05, 'Depreciation Rate (delta):', '.2f')
alpha_slider = create_slider(0, 1, 0.01, 0.35, 'Capital Output Elasticity (alpha):', '.2f')
A_slider = create_slider(0, 10, 1, 1, 'Initial Technology Level (A):', '.0f')
L_slider = create_slider(50, 150, 10, 100, 'Initial Labor Force (L):', '.0f')
K_slider = create_slider(5000, 15000, 1000, 10000, 'Initial Capital Stock (K):', '.0f')


# Group the sliders into a vertical box
ui = widgets.VBox([s_slider, n_slider, delta_slider, alpha_slider])

# Define the update function as before
def update_plot(s, n, delta, alpha, A, L, K):
    plot_potential_gdp(s=s, n=n, delta=delta, alpha=alpha, A=A, L=L, K=K)

# Create the interactive output widget
out = widgets.interactive_output(update_plot, {'s': s_slider, 'n': n_slider, 'delta': delta_slider, 'alpha': alpha_slider, 'A': A_slider, 'L': L_slider, 'K': K_slider})


# Display the UI and the output
display(ui, out)

Output()

## 4. Sensitivity Analysis: Technological Progress and Potential GDP

This section conducts a sensitivity analysis to explore the relationship between the rate of technological progress (`g`) and the potential Gross Domestic Product (GDP) growth rate.

### Economic Rationale
A central tenet of the Solow-Swan model is that sustained, long-term growth in per capita income is driven exclusively by technological progress. While factors like capital accumulation can lead to transitional growth, they are subject to diminishing returns. Technology, however, enhances the productivity of both capital and labor, allowing for continuous expansion of the production frontier.

This test is designed to empirically validate this theoretical conclusion within our simulated framework. By plotting the steady-state GDP growth rate across a range of values for `g`, we expect to see a direct, positive, and approximately linear relationship. This visualization powerfully illustrates why policies and economic conditions that foster innovation are paramount for achieving long-run prosperity.

### Test Specification
- The function `plot_potential_gdp` iterates through a predefined range of values for the technological progress rate (`g`).
- For each `g`, it calls the `solow_model_final_growth_rate` function to compute the corresponding steady-state growth rate.
- The resulting pairs of `(g, potential_gdp_growth)` are plotted, revealing the functional relationship between the two variables.
- Interactive widgets are again employed to allow for the dynamic adjustment of other background parameters (e.g., savings rate, depreciation), enabling a richer exploration of how these factors might mediate the `g`-to-GDP relationship.


In [5]:
# Solow model function as previously defined

# Define widgets for the Solow model parameters and CAPM inputs
s_widget = widgets.FloatSlider(value=0.05, min=0, max=1, step=0.01, description='Savings Rate (s):',style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.1%')
n_widget = widgets.FloatSlider(value=0.015, min=0, max=0.1, step=0.001, description='Population Growth Rate (n):', readout_format='.1%',style={'description_width': 'initial'}, layout=Layout(width='50%'))
g_widget = widgets.FloatSlider(value=0.016, min=0, max=0.1, step=0.001, description='Technological Growth Rate (g):', readout_format='.1%',style={'description_width': 'initial'}, layout=Layout(width='50%'))
delta_widget = widgets.FloatSlider(value=0.1, min=0, max=0.2, step=0.01, description='Depreciation Rate (delta):',style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.1%')
alpha_widget = widgets.FloatSlider(value=0.35, min=0, max=1, step=0.01, description='Capital Output Elasticity (alpha):',style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.1%')
beta_widget = widgets.FloatSlider(value=1, min=0, max=2, step=0.01, description='Beta:',style={'description_width': 'initial'}, layout=Layout(width='50%'))
equity_risk_premium_widget = widgets.FloatSlider(value=0.02, min=0, max=0.2, step=0.001, description='Equity Risk Premium:', readout_format='.2%',style={'description_width': 'initial'}, layout=Layout(width='50%'))
expected_inflation_widget = widgets.FloatSlider(value=0.02, min=0.01, max=0.05, step=0.001, description='Expected Inflation:', readout_format='.2%',style={'description_width': 'initial'}, layout=Layout(width='50%'))
term_premium_widget = widgets.FloatSlider(value=0.01, min=0, max=0.02, step=0.001, description='Term Premium:', readout_format='.2%',style={'description_width': 'initial'}, layout=Layout(width='50%'))
retention_rate_widget = widgets.FloatSlider(value=0.35, min=0, max=1, step=0.01, description='Retention Rate (b):',style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.1%')
tfp_widget = widgets.FloatSlider(value=1.1, min=0, max=5, step=0.01, description='Total Factor Productivity (tfp):', style={'description_width': 'initial'}, layout=Layout(width='50%'))

# Display widgets
display(s_widget, n_widget, g_widget, delta_widget, alpha_widget, beta_widget, equity_risk_premium_widget, expected_inflation_widget, term_premium_widget, retention_rate_widget,tfp_widget)

# Output widget for displaying results
output = widgets.Output()
display(output)
def calculate_required_return(risk_free_rate, beta, equity_risk_premium):
    return risk_free_rate + beta * (equity_risk_premium)


def update_justified_pe(*args):
    s = s_widget.value
    n = n_widget.value
    g = g_widget.value
    delta = delta_widget.value
    alpha = alpha_widget.value

    beta = beta_widget.value
    equity_risk_premium = equity_risk_premium_widget.value
    expected_inflation = expected_inflation_widget.value
    term_premium = term_premium_widget.value
    b = retention_rate_widget.value
    tfp = tfp_widget.value

    real_growth_rate = solow_model_final_growth_rate(s, n, g, delta, alpha)/100
    nominal_growth_rate = real_growth_rate + expected_inflation

    risk_free_rate = real_growth_rate + expected_inflation + term_premium
    required_return = calculate_required_return(risk_free_rate, beta, equity_risk_premium)

    earnings_growth_rate=nominal_growth_rate*tfp
    if nominal_growth_rate >= required_return:
        justified_pe_ratio = "Nominal growth rate cannot be equal to or greater than the required return."
    else:
        justified_pe_ratio = (1 - b) / (required_return - earnings_growth_rate)

    with output:
        output.clear_output()
        print(f"Justified Forward P/E Ratio: {justified_pe_ratio:.2f}")
        print(f"Real Growth Rate: {(100*real_growth_rate):.2f}")
        print(f"Nominal Growth Rate: {(100*nominal_growth_rate):.2f}")
        print(f"Risk Free Rate: {(100*risk_free_rate):.2f}")
        print(f"Required Return: {(100*required_return):.2f}")
        print(f"Earnings Growth Rate: {(100*earnings_growth_rate):.2f}")


# Attach the update function to all sliders
widgets_to_observe = [s_widget, n_widget, g_widget, delta_widget, alpha_widget, beta_widget, equity_risk_premium_widget, expected_inflation_widget, term_premium_widget, retention_rate_widget,tfp_widget]
for widget in widgets_to_observe:
    widget.observe(update_justified_pe, 'value')

update_justified_pe()  # Initialize the output

FloatSlider(value=0.05, description='Savings Rate (s):', layout=Layout(width='50%'), max=1.0, readout_format='…

FloatSlider(value=0.015, description='Population Growth Rate (n):', layout=Layout(width='50%'), max=0.1, reado…

FloatSlider(value=0.016, description='Technological Growth Rate (g):', layout=Layout(width='50%'), max=0.1, re…

FloatSlider(value=0.1, description='Depreciation Rate (delta):', layout=Layout(width='50%'), max=0.2, readout_…

FloatSlider(value=0.35, description='Capital Output Elasticity (alpha):', layout=Layout(width='50%'), max=1.0,…

...

FloatSlider(value=0.02, description='Expected Inflation:', layout=Layout(width='50%'), max=0.05, min=0.01, rea…

FloatSlider(value=0.01, description='Term Premium:', layout=Layout(width='50%'), max=0.02, readout_format='.2%…

FloatSlider(value=0.35, description='Retention Rate (b):', layout=Layout(width='50%'), max=1.0, readout_format…

FloatSlider(value=1.1, description='Total Factor Productivity (tfp):', layout=Layout(width='50%'), max=5.0, st…

Output()

## 5. Bridging Macroeconomics and Finance: The Justified P/E Ratio

This cell marks the crucial integration of our macroeconomic model with standard financial valuation theory to derive a fundamentally-grounded Price-to-Earnings (P/E) ratio.

### Economic and Financial Rationale
The value of an equity market is, in theory, the present discounted value of its future earnings. The P/E ratio is a widely used valuation metric that reflects this relationship. A "justified" P/E ratio is one that is consistent with the underlying economic and financial fundamentals. This model connects these fundamentals through a multi-step process grounded in established theory.

1.  **From Real to Nominal Growth**: The long-term **real growth rate** (`g_real`) is supplied by our Solow model (`n + g`). To value nominal cash flows, we convert this to a **nominal growth rate** (`g_nominal`) by adding the expected rate of inflation (`π`): \( g_{nominal} = g_{real} + π \).

2.  **Deriving the Risk-Free Rate**: The nominal risk-free rate (`R_f`) is modeled as the sum of the real economic growth rate, expected inflation, and a term premium (`TP`): \( R_f = g_{real} + π + TP \). This construction, consistent with the Fisher equation and theories of the term structure of interest rates, ensures that the risk-free rate reflects the underlying macroeconomic environment.

3.  **Calculating the Required Return on Equity**: We use the **Capital Asset Pricing Model (CAPM)** (Lintner, 1965; Sharpe, 1964) to determine the required rate of return on equity (`k_e`). This rate is the minimum return investors demand for bearing equity risk: \( k_e = R_f + β(ERP) \), where `β` is the market's systematic risk and `ERP` is the equity risk premium.

4.  **The Gordon Growth Model**: Finally, we use the constant-growth Dividend Discount Model, often referred to as the **Gordon Growth Model** (Gordon, 1959), to calculate the justified forward P/E ratio:
    \[ P/E = \frac{(1 - b)}{k_e - g_e} \]
    where `b` is the earnings retention rate (so `1-b` is the dividend payout ratio) and `g_e` is the expected growth rate of earnings, which is linked to the nominal economic growth rate.

This integrated approach ensures that the resulting P/E ratio is not an arbitrary number but is instead anchored to the productive capacity and long-run growth potential of the macroeconomy.

### Test Specification
- An interactive dashboard is constructed with widgets for all relevant Solow model and financial parameters.
- The `update_justified_pe` function orchestrates the four-step calculation described above.
- An `output` widget displays the calculated P/E ratio and all its constituent components in real time as the user adjusts the input parameters.

### References
- Gordon, M. J. (1959). Dividends, Earnings, and Stock Prices. *The Review of Economics and Statistics, 41*(2), 99–105.
- Lintner, J. (1965). The Valuation of Risk Assets and the Selection of Risky Investments in Stock Portfolios and Capital Budgets. *The Review of Economics and Statistics, 47*(1), 13–37.
- Sharpe, W. F. (1964). Capital Asset Prices: A Theory of Market Equilibrium under Conditions of Risk. *The Journal of Finance, 19*(3), 425–442.


In [7]:
# Assuming the solow_model_final_growth_rate and calculate_required_return functions are defined

# Define widgets for the Solow model parameters and CAPM inputs
s_widget = widgets.FloatSlider(value=0.05, min=0, max=1, step=0.01, description='Savings Rate (s):',style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.2%')
n_widget = widgets.FloatSlider(value=0.015, min=0, max=0.1, step=0.001, description='Population Growth Rate (n):', readout_format='.1%',style={'description_width': 'initial'}, layout=Layout(width='50%'),)
delta_widget = widgets.FloatSlider(value=0.1, min=0, max=0.2, step=0.01, description='Depreciation Rate (delta):',style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.2%')
alpha_widget = widgets.FloatSlider(value=0.35, min=0, max=1, step=0.01, description='Capital Output Elasticity (alpha):',style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.2%')
beta_widget = widgets.FloatSlider(value=1, min=0, max=2, step=0.01, description='Beta:',style={'description_width': 'initial'}, layout=Layout(width='50%'))
equity_risk_premium_widget = widgets.FloatSlider(value=0.02, min=0, max=0.2, step=0.001, description='Equity Risk Premium:', readout_format='.2%',style={'description_width': 'initial'}, layout=Layout(width='50%'))
expected_inflation_widget = widgets.FloatSlider(value=0.02, min=0.01, max=0.05, step=0.001, description='Expected Inflation:', readout_format='.2%',style={'description_width': 'initial'}, layout=Layout(width='50%'))
term_premium_widget = widgets.FloatSlider(value=0.01, min=0, max=0.02, step=0.001, description='Term Premium:', readout_format='.2%',style={'description_width': 'initial'}, layout=Layout(width='50%'))
retention_rate_widget = widgets.FloatSlider(value=0.35, min=0, max=1, step=0.01, description='Retention Rate (b):',style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.2%')
tfp_widget = widgets.FloatSlider(value=1.1, min=0, max=5, step=0.1, description='Total Factor Productivity (tfp):', style={'description_width': 'initial'}, layout=Layout(width='50%'))


# Display widgets
display(s_widget, n_widget, delta_widget, alpha_widget, beta_widget, equity_risk_premium_widget, expected_inflation_widget, term_premium_widget, retention_rate_widget, tfp_widget)

# Output widget for displaying results and plot
output = widgets.Output()
display(output)

def plot_pe_ratio(change=None):  # Add 'change' parameter to accept the event object
    g_values = np.arange(0.015, 0.031, 0.0005)  # Technological growth rate from 1.5% to 2.5%
    pe_ratios = []

    for g in g_values:
        s = s_widget.value
        n = n_widget.value
        delta = delta_widget.value
        alpha = alpha_widget.value
        beta = beta_widget.value
        equity_risk_premium = equity_risk_premium_widget.value
        expected_inflation = expected_inflation_widget.value
        term_premium = term_premium_widget.value
        b = retention_rate_widget.value
        tfp = tfp_widget.value

        real_growth_rate = solow_model_final_growth_rate(s, n, g, delta, alpha) / 100
        nominal_growth_rate = real_growth_rate + expected_inflation

        risk_free_rate = real_growth_rate + expected_inflation + term_premium
        required_return = calculate_required_return(risk_free_rate, beta, equity_risk_premium)

        earnings_growth_rate = nominal_growth_rate * tfp
        if nominal_growth_rate >= required_return:
            pe_ratio = np.nan  # Use NaN for cases where the calculation doesn't make sense
        else:
            pe_ratio = (1 - b) / (required_return - earnings_growth_rate)
        pe_ratios.append(pe_ratio)

    with output:
        clear_output(wait=True)
        plt.figure(figsize=(10, 6))
        plt.plot(g_values * 100, pe_ratios, marker='o', linestyle='-', color='blue')
        for i, txt in enumerate(pe_ratios):# Check if index is even and value is not NaN
            if not np.isnan(txt) and i % 4 == 0:
                plt.text(g_values[i] * 100, txt, f'{txt:.2f}', fontsize=8, fontweight='bold', ha='right', va='top')
        plt.title('Justified Forward P/E Ratio vs. Technological Growth Rate')
        plt.xlabel('Technological Growth Rate (%)')
        plt.ylabel('Justified Forward P/E Ratio')
        plt.grid(True)
        plt.show()

# Attach the plot_pe_ratio function to update whenever a widget value changes
widgets_to_observe = [s_widget, n_widget, delta_widget, alpha_widget, beta_widget, equity_risk_premium_widget, expected_inflation_widget, term_premium_widget, retention_rate_widget, tfp_widget]
for widget in widgets_to_observe:
    widget.observe(plot_pe_ratio, 'value')

# Initial call to display the plot with the default values
plot_pe_ratio()

FloatSlider(value=0.05, description='Savings Rate (s):', layout=Layout(width='50%'), max=1.0, readout_format='…

FloatSlider(value=0.015, description='Population Growth Rate (n):', layout=Layout(width='50%'), max=0.1, reado…

FloatSlider(value=0.1, description='Depreciation Rate (delta):', layout=Layout(width='50%'), max=0.2, readout_…

FloatSlider(value=0.35, description='Capital Output Elasticity (alpha):', layout=Layout(width='50%'), max=1.0,…

FloatSlider(value=1.0, description='Beta:', layout=Layout(width='50%'), max=2.0, step=0.01, style=SliderStyle(…

...

FloatSlider(value=0.02, description='Expected Inflation:', layout=Layout(width='50%'), max=0.05, min=0.01, rea…

FloatSlider(value=0.01, description='Term Premium:', layout=Layout(width='50%'), max=0.02, readout_format='.2%…

FloatSlider(value=0.35, description='Retention Rate (b):', layout=Layout(width='50%'), max=1.0, readout_format…

FloatSlider(value=1.1, description='Total Factor Productivity (tfp):', layout=Layout(width='50%'), max=5.0, st…

Output()

## 6. Sensitivity Analysis: P/E Ratio and Technological Growth

This cell investigates the sensitivity of the justified P/E ratio to changes in the rate of technological progress (`g`).

### Rationale
As established, technological progress (`g`) is the ultimate driver of long-run real economic growth. Since the growth rate of earnings (`g_e`) is a key input in the Gordon Growth Model, `g` should have a powerful influence on the justified P/E ratio. All else being equal, a higher `g` implies higher future earnings growth, which should command a higher valuation multiple.

This analysis is critical for understanding the market's sensitivity to shifts in long-term growth expectations. It helps quantify how much of a market's valuation is predicated on assumptions about future innovation and productivity.

### Test Specification
- The `plot_pe_ratio` function calculates the justified P/E ratio for a range of `g` values, holding other variables constant.
- The results are plotted to visualize the functional relationship between technological growth and the P/E ratio.
- The entire process is embedded in an interactive widget framework. This allows the user to change other parameters (like the equity risk premium or inflation) and immediately see how the sensitivity curve shifts in response, providing a deep, multi-dimensional view of the valuation model. For example, one can observe that in a higher risk-premium environment, the P/E ratio is lower for any given level of `g`.


In [8]:
# Number of simulations
num_simulations = 1000
num_months = 12

# Assumed mean and standard deviation for the inputs
# Example: Technological growth rate (annualized, in decimals)
g_mean = 0.02
g_std = 0.005
# Expected inflation (annualized, in decimals)
inflation_mean = 0.02
inflation_std = 0.01
# Term premium (annualized, in decimals)
term_premium_mean = 0.005
term_premium_std = 0.002

# Other fixed inputs
s = 0.20
n = 0.005
delta = 0.05
alpha = 0.35
beta = 1
equity_risk_premium = 0.05
b = 0.6
tfp = 2

def calculate_pe_ratio(g, inflation, term_premium):
    real_growth_rate = solow_model_final_growth_rate(s, n, g, delta, alpha) / 100
    nominal_growth_rate = real_growth_rate + inflation
    risk_free_rate = real_growth_rate + inflation + term_premium
    required_return = calculate_required_return(risk_free_rate, beta, equity_risk_premium)
    earnings_growth_rate = nominal_growth_rate * tfp
    if nominal_growth_rate >= required_return:
        return np.nan
    else:
        return (1 - b) / (required_return - earnings_growth_rate), earnings_growth_rate



# Function to run Monte Carlo simulation
def run_monte_carlo_simulation(g_mean, g_std, inflation_mean, inflation_std, term_premium_mean, term_premium_std, s, n, delta, alpha, beta, equity_risk_premium, b, tfp, num_simulations=10000, num_months=1):
    pe_ratios = np.zeros((num_simulations, num_months))
    earnings_growth_rates = np.zeros((num_simulations, num_months))

    for i in range(num_simulations):
        for month in range(num_months):
            g = np.random.normal(g_mean, g_std)
            inflation = np.random.normal(inflation_mean, inflation_std)
            term_premium = np.random.normal(term_premium_mean, term_premium_std)

            pe_ratio, earnings_growth_rate = calculate_pe_ratio(g, inflation, term_premium)
            pe_ratios[i, month] = pe_ratio
            earnings_growth_rates[i, month] = earnings_growth_rate

    return pe_ratios, earnings_growth_rates


# Define the required widgets for the inputs
s_widget = widgets.FloatSlider(value=0.05, min=0, max=0.2, step=0.01, description='Savings Rate (s):', style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.2%')
n_widget = widgets.FloatSlider(value=0.011, min=0, max=0.03, step=0.001, description='Population Growth Rate (n):', style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.2%')
delta_widget = widgets.FloatSlider(value=0.05, min=0.01, max=0.1, step=0.01, description='Depreciation Rate (delta):', style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.2%')
alpha_widget = widgets.FloatSlider(value=0.35, min=0.25, max=0.5, step=0.05, description='Capital Output Elasticity (alpha):', style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.2%')
beta_widget = widgets.FloatSlider(value=1, min=0.5, max=1.5, step=0.1, description='Beta:', style={'description_width': 'initial'}, layout=Layout(width='50%'))
equity_risk_premium_widget = widgets.FloatSlider(value=0.025, min=0.01, max=0.1, step=0.001, description='Equity Risk Premium:', style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.2%')
b_widget = widgets.FloatSlider(value=0.35, min=0.1, max=0.6, step=0.05, description='Retention Rate (b):', style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.2%')
tfp_widget = widgets.FloatSlider(value=1.1, min=1, max=2, step=0.1, description='Earnings Growth Factor:', style={'description_width': 'initial'}, layout=Layout(width='50%'))
# Widgets for the variables with assumed distributions
g_mean_widget = widgets.FloatSlider(value=0.015, min=0.01, max=0.03, step=0.001, description='Technological Growth Rate Mean (g_mean):', style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.2%')
g_std_widget = widgets.FloatSlider(value=0.0005, min=0, max=0.001, step=0.0001, description='Technological Growth Rate Std Dev (g_std):', style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.4%')
inflation_mean_widget = widgets.FloatSlider(value=0.02, min=0.01, max=0.03, step=0.001, description='Inflation Mean:', style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.2%')
inflation_std_widget = widgets.FloatSlider(value=0.0001, min=0, max=0.001, step=0.0001, description='Inflation Std Dev:', style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.4%')
term_premium_mean_widget = widgets.FloatSlider(value=0.005, min=0.001, max=0.02, step=0.001, description='Term Premium Mean:', style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.2%')
term_premium_std_widget = widgets.FloatSlider(value=0.0001, min=0, max=0.001, step=0.0001, description='Term Premium Std Dev:', style={'description_width': 'initial'}, layout=Layout(width='50%'),readout_format='.4%')

# Output widget for the plot
output = widgets.Output()

# Display widgets
display_widgets = [s_widget, n_widget, delta_widget, alpha_widget, beta_widget, equity_risk_premium_widget, b_widget, tfp_widget,
                   g_mean_widget, g_std_widget, inflation_mean_widget, inflation_std_widget, term_premium_mean_widget, term_premium_std_widget]
for widget in display_widgets:
    display(widget)

display(output)

# This function should encapsulate the previously shown Monte Carlo simulation logic, adjusted to use widget values.
def update_plot(change):
    pe_ratios, earnings_growth_rates = run_monte_carlo_simulation(
        g_mean=g_mean_widget.value,
        g_std=g_std_widget.value,
        inflation_mean=inflation_mean_widget.value,
        inflation_std=inflation_std_widget.value,
        term_premium_mean=term_premium_mean_widget.value,
        term_premium_std=term_premium_std_widget.value,
        s=s_widget.value,
        n=n_widget.value,
        delta=delta_widget.value,
        alpha=alpha_widget.value,
        beta=beta_widget.value,
        equity_risk_premium=equity_risk_premium_widget.value,
        b=b_widget.value,
        tfp=tfp_widget.value
    )

    final_month_pe_ratios = pe_ratios[:, -1][~np.isnan(pe_ratios[:, -1])]  # Filter out NaNs
    final_month_earnings_growth_rates = earnings_growth_rates[:, -1][~np.isnan(earnings_growth_rates[:, -1])] * 100  # Convert to percentage and filter out NaNs

    pe_mean, pe_std = np.mean(final_month_pe_ratios), np.std(final_month_pe_ratios)
    earnings_mean, earnings_std = np.mean(final_month_earnings_growth_rates), np.std(final_month_earnings_growth_rates)


    with output:
        clear_output(wait=True)
        fig, axs = plt.subplots(1, 2, figsize=(15, 4))

        axs[0].hist(final_month_pe_ratios, bins=30, alpha=0.75, color='blue')
        axs[0].axvline(pe_mean, color='k', linestyle='dashed', linewidth=1)
        axs[0].axvline(pe_mean + 2 * pe_std, color='r', linestyle='dashed', linewidth=1)
        axs[0].axvline(pe_mean - 2 * pe_std, color='r', linestyle='dashed', linewidth=1)
        axs[0].set_title('Distribution of Simulated P/E Ratios')
        axs[0].set_xlabel('P/E Ratio')
        axs[0].set_ylabel('Frequency')
        axs[0].grid(True)

        # Plot Earnings Growth Rates
        axs[1].hist(final_month_earnings_growth_rates, bins=30, alpha=0.75, color='green')
        axs[1].axvline(earnings_mean, color='k', linestyle='dashed', linewidth=1)
        axs[1].axvline(earnings_mean + 2 * earnings_std, color='r', linestyle='dashed', linewidth=1)
        axs[1].axvline(earnings_mean - 2 * earnings_std, color='r', linestyle='dashed', linewidth=1)
        axs[1].set_title('Distribution of Simulated Earnings Growth Rates')
        axs[1].set_xlabel('Earnings Growth Rate (%)')
        axs[1].set_ylabel('Frequency')
        axs[1].grid(True)

        plt.tight_layout()
        plt.show()

# Trigger the update_plot function whenever any widget's value changes
for widget in display_widgets:
    widget.observe(update_plot, names='value')

# Initial call to display the plot with default widget values
update_plot(None)

FloatSlider(value=0.05, description='Savings Rate (s):', layout=Layout(width='50%'), max=0.2, readout_format='…

FloatSlider(value=0.011, description='Population Growth Rate (n):', layout=Layout(width='50%'), max=0.03, read…

FloatSlider(value=0.05, description='Depreciation Rate (delta):', layout=Layout(width='50%'), max=0.1, min=0.0…

FloatSlider(value=0.35, description='Capital Output Elasticity (alpha):', layout=Layout(width='50%'), max=0.5,…

FloatSlider(value=1.0, description='Beta:', layout=Layout(width='50%'), max=1.5, min=0.5, style=SliderStyle(de…

...

FloatSlider(value=0.02, description='Inflation Mean:', layout=Layout(width='50%'), max=0.03, min=0.01, readout…

FloatSlider(value=0.0001, description='Inflation Std Dev:', layout=Layout(width='50%'), max=0.001, readout_for…

FloatSlider(value=0.005, description='Term Premium Mean:', layout=Layout(width='50%'), max=0.02, min=0.001, re…

FloatSlider(value=0.0001, description='Term Premium Std Dev:', layout=Layout(width='50%'), max=0.001, readout_…

Output()

## 7. Stochastic Analysis: A Monte Carlo Simulation of Asset Valuation

This final section advances the analysis from a deterministic framework to a stochastic one by employing a Monte Carlo simulation. This approach allows us to model and quantify the impact of uncertainty in key economic variables on equity market valuation.

### Rationale
Deterministic models, while useful, produce single point estimates that can create a false sense of precision. In reality, economic variables are not known with certainty. Key drivers of our valuation model—specifically the long-run rate of technological progress (`g`), the rate of inflation (`π`), and the term premium (`TP`)—are unobservable and subject to significant uncertainty.

Monte Carlo simulation is a computational technique that addresses this issue by running a large number of trials, or "simulations," to model the probability of different outcomes. Instead of assuming a single value for each uncertain variable, we assume a probability distribution (in this case, a normal distribution) from which values are randomly drawn in each simulation. By running thousands of these simulations, we can build a distribution of the potential justified P/E ratios and earnings growth rates, providing a much richer understanding of the range of plausible outcomes and the risks involved.

### Test Specification
- **Assumed Distributions**: We model `g`, `π`, and `TP` as random variables drawn from normal distributions, defined by a mean and a standard deviation. These parameters can be adjusted via interactive widgets.
- **Simulation Loop**: The `run_monte_carlo_simulation` function performs a loop for a specified number of simulations (`num_simulations`). In each iteration, it:
    1. Draws random values for `g`, `π`, and `TP`.
    2. Recalculates the entire macro-financial linkage to arrive at a single justified P/E ratio for that specific draw.
- **Aggregation and Visualization**: After all simulations are complete, the model collects the results and plots histograms of the distribution of the simulated P/E ratios and earnings growth rates. The mean, standard deviation, and confidence intervals are calculated and displayed, providing a statistical summary of the valuation under uncertainty. This transforms the valuation exercise from "What is the P/E ratio?" to "What is the probable range of the P/E ratio, given our uncertainty about the future?".


## 8. Conclusion

This notebook has demonstrated the construction and application of a fully integrated macro-financial model. By systematically linking the real economic growth engine of the Solow-Swan model to the valuation principles of the CAPM and Gordon Growth Model, we have created a tool that grounds equity market valuation in fundamental economic theory.

The analysis progressed through several stages of increasing complexity:
1.  **Core Model Specification**: Defining the fundamental equations of economic growth.
2.  **Interactive Exploration**: Building intuitive dashboards to understand model dynamics.
3.  **Sensitivity Analysis**: Quantifying the impact of key parameters, particularly technological progress, on both growth and valuation.
4.  **Stochastic Simulation**: Moving beyond deterministic estimates to a probabilistic framework that explicitly models uncertainty.

The results reaffirm key theoretical insights: the paramount importance of technological progress for long-term growth and valuations, and the sensitivity of asset prices to changes in fundamental economic conditions and risk perceptions. The framework presented here is both extensible and practical, serving as a robust foundation for further research in macro-financial modeling and applied asset allocation.
